Даны данные по пациентам, проходящим диспансеризацию в мед клинике. По результатам диспансеризации надо построить ML модель, которая определит вероятность возникновения сердечно-сосудистого заболевания у пациент.

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

import warnings
from pandas.core.common import SettingWithCopyWarning

import optuna

Импортировал нужные библиотеки

Далее проведу обзор данных

In [8]:
train = pd.read_csv('/kaggle/input/yap9-heart-diseases-prediction/train.csv')
test = pd.read_csv('/kaggle/input/yap9-heart-diseases-prediction/test.csv')
display(train)
display(test)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,5,18888,1,154,85.0,130,80,1,1,0,0,1
1,6,19042,2,170,69.0,130,90,1,1,0,0,1
2,7,20432,1,160,70.0,120,75,1,1,0,0,0
3,10,18133,2,185,94.0,130,90,1,1,0,0,1
4,11,16093,2,170,76.0,120,80,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99984,21722,2,170,103.0,150,90,1,1,0,1,0
29996,99987,18309,1,172,78.0,110,70,1,1,0,0,1
29997,99989,21247,2,169,61.0,120,80,1,1,0,0,0
29998,99994,14509,2,175,80.0,120,80,2,1,0,0,0


In [9]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
test.info()
test.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           30000 non-null  int64  
 1   age          30000 non-null  int64  
 2   gender       30000 non-null  int64  
 3   height       30000 non-null  int64  
 4   weight       30000 non-null  float64
 5   ap_hi        30000 non-null  int64  
 6   ap_lo        30000 non-null  int64  
 7   cholesterol  30000 non-null  int64  
 8   gluc         30000 non-null  int64  
 9   smoke        30000 non-null  int64  
 10  alco         30000 non-null  int64  
 11  active       30000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 2.7 MB


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.00000,30000.000000,30000.000000
mean,50062.686900,19455.528400,1.353733,164.322267,74.120615,130.271300,95.689000,1.368033,1.222700,0.08860,0.055000,0.805733
std,28906.167148,2477.840922,0.478136,8.213740,14.341540,212.501152,163.852598,0.679166,0.566494,0.28417,0.227984,0.395642
min,5.000000,8865.000000,1.000000,50.000000,10.000000,-130.000000,-90.000000,1.000000,1.000000,0.00000,0.000000,0.000000
25%,24983.250000,17658.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.00000,0.000000,1.000000
50%,49995.500000,19694.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.00000,0.000000,1.000000
75%,75232.250000,21320.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.00000,0.000000,1.000000
max,99997.000000,23705.000000,2.000000,198.000000,183.000000,16020.000000,9100.000000,3.000000,3.000000,1.00000,1.000000,1.000000


Как видно, в столбцах встречаются завышенные и отрицательные значения по столбцам ap_hi(верхнее давление) и ap_lo(нижнее давление)
Так же ясно, что возраст представлен в днях.

In [11]:
train_repaired = train
train_repaired.loc[train_repaired['ap_hi'] > 10000,'ap_hi'] = train_repaired.loc[train_repaired['ap_hi'] > 10000,'ap_hi']/100
train_repaired.loc[train_repaired['ap_lo'] > 10000,'ap_lo'] = train_repaired.loc[train_repaired['ap_lo'] > 10000,'ap_lo']/100
train_repaired.loc[train_repaired['ap_hi'] > 400,'ap_hi'] = train_repaired.loc[train_repaired['ap_hi'] > 400,'ap_hi']/10
train_repaired.loc[train_repaired['ap_lo'] > 400,'ap_lo'] = train_repaired.loc[train_repaired['ap_lo'] > 400,'ap_lo']/10
train_repaired = abs(train_repaired)
train_repaired['age'] = train_repaired['age']/365
train_repaired['age'] = train_repaired['age'].round()
display(train_repaired.describe())

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,53.338686,1.349571,164.359229,74.205690,126.702566,81.845080,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,6.765294,0.476838,8.210126,14.395757,18.060211,16.946128,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,30.000000,1.000000,55.000000,10.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,58.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,65.000000,2.000000,250.000000,200.000000,309.000000,1000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
train_repaired['imt'] = train_repaired['weight'] / train_repaired['height']
train_repaired['imt'] = train_repaired['imt'].round(3)
display(train_repaired)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,imt
0,0,50.0,2,168,62.0,110.0,80.0,1,1,0,0,1,0,0.369
1,1,55.0,1,156,85.0,140.0,90.0,3,1,0,0,1,1,0.545
2,2,52.0,1,165,64.0,130.0,70.0,3,1,0,0,0,1,0.388
3,3,48.0,2,169,82.0,150.0,100.0,1,1,0,0,1,1,0.485
4,4,48.0,1,156,56.0,100.0,60.0,1,1,0,0,0,0,0.359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,53.0,2,168,76.0,120.0,80.0,1,1,1,0,1,0,0.452
69996,99995,62.0,1,158,126.0,140.0,90.0,2,2,0,0,1,1,0.797
69997,99996,52.0,2,183,105.0,180.0,90.0,3,1,0,1,0,1,0.574
69998,99998,61.0,1,163,72.0,135.0,80.0,1,2,0,0,0,1,0.442


Создал столбец индекса массы тела, который пригодится в обучении моделей.

In [13]:
test_repaired = test
test_repaired.loc[test_repaired['ap_hi'] > 10000,'ap_hi'] = test_repaired.loc[test_repaired['ap_hi'] > 10000,'ap_hi']/100
test_repaired.loc[test_repaired['ap_lo'] > 10000,'ap_lo'] = test_repaired.loc[test_repaired['ap_lo'] > 10000,'ap_lo']/100
test_repaired.loc[test_repaired['ap_hi'] > 400,'ap_hi'] = test_repaired.loc[test_repaired['ap_hi'] > 400,'ap_hi']/10
test_repaired.loc[test_repaired['ap_lo'] > 400,'ap_lo'] = test_repaired.loc[test_repaired['ap_lo'] > 400,'ap_lo']/10
test_repaired = abs(test_repaired)
test_repaired['age'] = test_repaired['age']/365
test_repaired['age'] = test_repaired['age'].round()
display(test_repaired.describe())

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.00000,30000.000000,30000.000000
mean,50062.686900,53.305467,1.353733,164.322267,74.120615,126.720103,81.818140,1.368033,1.222700,0.08860,0.055000,0.805733
std,28906.167148,6.794338,0.478136,8.213740,14.341540,18.074236,15.062881,0.679166,0.566494,0.28417,0.227984,0.395642
min,5.000000,24.000000,1.000000,50.000000,10.000000,1.000000,0.000000,1.000000,1.000000,0.00000,0.000000,0.000000
25%,24983.250000,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.00000,0.000000,1.000000
50%,49995.500000,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.00000,0.000000,1.000000
75%,75232.250000,58.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.00000,0.000000,1.000000
max,99997.000000,65.000000,2.000000,198.000000,183.000000,400.000000,910.000000,3.000000,3.000000,1.00000,1.000000,1.000000


Исправил ошибки в данных

In [14]:
test_repaired['imt'] = test_repaired['weight']/test_repaired['height']
test_repaired['imt'] = test_repaired['imt'].round(3)
display(test_repaired)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,imt
0,5,52.0,1,154,85.0,130.0,80.0,1,1,0,0,1,0.552
1,6,52.0,2,170,69.0,130.0,90.0,1,1,0,0,1,0.406
2,7,56.0,1,160,70.0,120.0,75.0,1,1,0,0,0,0.438
3,10,50.0,2,185,94.0,130.0,90.0,1,1,0,0,1,0.508
4,11,44.0,2,170,76.0,120.0,80.0,1,1,0,0,1,0.447
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99984,60.0,2,170,103.0,150.0,90.0,1,1,0,1,0,0.606
29996,99987,50.0,1,172,78.0,110.0,70.0,1,1,0,0,1,0.453
29997,99989,58.0,2,169,61.0,120.0,80.0,1,1,0,0,0,0.361
29998,99994,40.0,2,175,80.0,120.0,80.0,2,1,0,0,0,0.457


In [15]:
df_train, df_valid = train_test_split(train_repaired, test_size=0.19, random_state=12345) # отделите 25% данных для валидационной выборки

features_train = df_train.drop(['id','cardio'], axis=1)
target_train = df_train['cardio']
features_valid = df_valid.drop(['id','cardio'], axis=1)
target_valid = df_valid['cardio']

Разбил тренировочный датафрейм на 2 выборки - Тренировочную и валидационную

In [16]:
features_train = np.array(features_train)
target_train = np.array(target_train)
features_valid = np.array(features_valid)
target_valid = np.array(target_valid)


In [17]:
cat_features = list(range(0, features_train.shape[1]))
clf = CatBoostClassifier(iterations = 2000, random_seed=12345,
    learning_rate=0.07, border_count = 254, objective = "Logloss", colsample_bylevel = 0.07866549011928872,
                        depth = 6, boosting_type = "Ordered",
                        bootstrap_type = "Bernoulli", subsample = 0.5961285591174476)
clf.fit(features_train, target_train)
predictions_val2 = clf.predict_proba(features_valid)[:,1] 


0:	learn: 0.6931470	total: 73.5ms	remaining: 2m 26s
1:	learn: 0.6912787	total: 95.4ms	remaining: 1m 35s
2:	learn: 0.6884547	total: 115ms	remaining: 1m 16s
3:	learn: 0.6747184	total: 139ms	remaining: 1m 9s
4:	learn: 0.6631613	total: 161ms	remaining: 1m 4s
5:	learn: 0.6519013	total: 194ms	remaining: 1m 4s
6:	learn: 0.6519013	total: 211ms	remaining: 1m
7:	learn: 0.6500758	total: 229ms	remaining: 56.9s
8:	learn: 0.6488548	total: 248ms	remaining: 54.9s
9:	learn: 0.6488542	total: 264ms	remaining: 52.6s
10:	learn: 0.6488482	total: 288ms	remaining: 52.1s
11:	learn: 0.6402747	total: 308ms	remaining: 51s
12:	learn: 0.6394147	total: 326ms	remaining: 49.8s
13:	learn: 0.6379310	total: 345ms	remaining: 49s
14:	learn: 0.6379304	total: 361ms	remaining: 47.8s
15:	learn: 0.6362021	total: 378ms	remaining: 46.9s
16:	learn: 0.6249918	total: 411ms	remaining: 47.9s
17:	learn: 0.6249910	total: 427ms	remaining: 47s
18:	learn: 0.6249903	total: 442ms	remaining: 46.1s
19:	learn: 0.6249897	total: 458ms	remaining: 

Использовал модель CatBoost как наиболее точную

In [18]:
result2 = roc_auc_score(target_valid, predictions_val2)
print(result2)

0.8092354971380114


roc_auc_score для определения точности модели

In [19]:
test_id = test_repaired['id']
display(test_id)
test_repaired = test_repaired.drop(['id'], axis=1)

0            5
1            6
2            7
3           10
4           11
         ...  
29995    99984
29996    99987
29997    99989
29998    99994
29999    99997
Name: id, Length: 30000, dtype: int64

In [20]:
predict_fin_clf = clf.predict_proba(test_repaired)[:,1]
final_clf = test_repaired
final_clf['cardio'] = predict_fin_clf
final_clf['id'] = test_id
display(final_clf)
final2_clf = final_clf[['id','cardio']]
display(final2_clf)
display(final2_clf.describe())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,imt,cardio,id
0,52.0,1,154,85.0,130.0,80.0,1,1,0,0,1,0.552,0.473585,5
1,52.0,2,170,69.0,130.0,90.0,1,1,0,0,1,0.406,0.530484,6
2,56.0,1,160,70.0,120.0,75.0,1,1,0,0,0,0.438,0.392661,7
3,50.0,2,185,94.0,130.0,90.0,1,1,0,0,1,0.508,0.541141,10
4,44.0,2,170,76.0,120.0,80.0,1,1,0,0,1,0.447,0.217005,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,60.0,2,170,103.0,150.0,90.0,1,1,0,1,0,0.606,0.881133,99984
29996,50.0,1,172,78.0,110.0,70.0,1,1,0,0,1,0.453,0.178547,99987
29997,58.0,2,169,61.0,120.0,80.0,1,1,0,0,0,0.361,0.404600,99989
29998,40.0,2,175,80.0,120.0,80.0,2,1,0,0,0,0.457,0.257798,99994


,id,cardio
0,5,0.473585
1,6,0.530484
2,7,0.392661
3,10,0.541141
4,11,0.217005
...,...,...
29995,99984,0.881133
29996,99987,0.178547
29997,99989,0.404600
29998,99994,0.257798


,id,cardio
count,30000.000000,30000.000000
mean,50062.686900,0.498201
std,28906.167148,0.263144
min,5.000000,0.016073
25%,24983.250000,0.266850
50%,49995.500000,0.456211
75%,75232.250000,0.795137
max,99997.000000,0.971719


Применил модель к тестовой выборке, по которой и требовалось предсказать

In [21]:
final2_clf.to_csv('maxvolkov_clf8.csv',index=False)